# **3368. First Letter Capitalization**

**Table: user_content**
``` sql
+-------------+---------+
| Column Name | Type    |
+-------------+---------+
| content_id  | int     |
| content_text| varchar |
+-------------+---------+
```
content_id is the unique key for this table.

Each row contains a unique ID and the corresponding text content.

Write a solution to transform the text in the content_text column by applying the following rules:

Convert the first letter of each word to uppercase

Keep all other letters in lowercase

Preserve all existing spaces

Note: There will be no special character in content_text.

Return the result table that includes both the original content_text and the modified text where each word starts with a capital letter.

The result format is in the following example.

 

**Example:**

**Input:**

**user_content table:**
``` sql
+------------+-----------------------------------+
| content_id | content_text                      |
+------------+-----------------------------------+
| 1          | hello world of SQL                |
| 2          | the QUICK brown fox               |
| 3          | data science AND machine learning |
| 4          | TOP rated programming BOOKS       |
+------------+-----------------------------------+
```
**Output:**
``` sql
+------------+-----------------------------------+-----------------------------------+
| content_id | original_text                     | converted_text                    |
+------------+-----------------------------------+-----------------------------------+
| 1          | hello world of SQL                | Hello World Of Sql                |
| 2          | the QUICK brown fox               | The Quick Brown Fox               |
| 3          | data science AND machine learning | Data Science And Machine Learning |
| 4          | TOP rated programming BOOKS       | Top Rated Programming Books       |
+------------+-----------------------------------+-----------------------------------+
```
**Explanation:**

**For content_id = 1:**
- Each word's first letter is capitalized: Hello World Of Sql

**For content_id = 2:**
- Original mixed-case text is transformed to title case: The Quick Brown Fox

**For content_id = 3:**
- The word AND is converted to "And": "Data Science And Machine Learning"

**For content_id = 4:**
- Handles word TOP rated correctly: Top Rated
- Converts BOOKS from all caps to title case: Books

**Solution-1:**
``` sql
WITH RECURSIVE cte AS (
    -- Base case: Extract the first word and remaining text
    SELECT 
        content_id,content_text,
        TRIM(REGEXP_SUBSTR(content_text, '^[a-zA-Z]+')) AS word,
        TRIM(REGEXP_REPLACE(content_text, '^[a-zA-Z]+', '')) AS remain,
        1 as rn
    FROM user_content

    UNION ALL

    -- Recursive case: Continue extracting words from the remaining text
    SELECT 
        content_id,content_text,
        TRIM(REGEXP_SUBSTR(remain, '^[a-zA-Z]+')) AS word,
        TRIM(REGEXP_REPLACE(remain, '^[a-zA-Z]+', '')) AS remain,
        rn+1
    FROM cte
    WHERE remain <> '' -- Stop when no more text remains
)

,cte2 as(
SELECT content_id, content_text,CONCAT(UPPER(LEFT(word, 1)), LOWER(SUBSTRING(word, 2))) as word, rn
FROM cte
WHERE word IS NOT NULL)

select content_id, content_text as original_text,
group_concat(word order by rn separator ' ') as converted_text
from cte2
group by content_id
order by content_id;